In [1]:
# root_path = 'C:\\Users\\hyuns\\Desktop\\캡스톤\\EEGData\\VR300\\실험데이터\\S12\\'
root_path = 'C:\\Users\\hyuns\\Desktop\\캡스톤\\EEGData\\VR300\\0729\\hs\\'

# filtering bandwidth
low, high = 0.1, 30

resampleRate = 100

In [2]:
import numpy as np
import pickle
import pandas as pd
import statsmodels.api as sm
import hdf5storage
from scipy.signal import butter, lfilter
import os, glob, time
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from datetime import datetime
from sklearn.externals import joblib
from scipy import signal
import matplotlib.pyplot as plt
import random

def Re_referencing(eegData, channelNum, sampleNum):
        after_car = np.zeros((channelNum,sampleNum))
        for i in np.arange(channelNum):
            after_car[i,:] = eegData[i,:] - np.mean(eegData,axis=0)
        return after_car
    
def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

def Epoching(eegData, stims, code, samplingRate, nChannel, epochSampleNum, epochOffset,baseline):
        Time = stims[np.where(stims[:,1] == code),0][0]
        Time = np.floor(np.multiply(Time,samplingRate)).astype(int)
        Time_after = np.add(Time,epochOffset).astype(int)
        
        
        base = -90
        Time_base = np.add(Time,base).astype(int)
        
        
        Num = Time.shape
        Epochs = np.zeros((Num[0], nChannel, epochSampleNum))
        
        for j in range(Num[0]):
            Epochs[j, :, :] = eegData[:, Time_after[j]:Time_after[j] + epochSampleNum]
            
            for i in range(nChannel):
#                 Epochs[j,i,:] = Epochs[j,i,:] - np.mean(Epochs[j,i,:])
                Epochs[j,i,:] = Epochs[j,i,:] - np.mean(eegData[:, Time_base[j]:Time[j]])
                
                
        return [Epochs, Num[0]]

def Convert_to_featureVector(EpochsT, NumT, EpochsN, NumN, featureNum):
        FeaturesT = np.zeros((NumT, featureNum))
        for i in range(NumT):
            FeaturesT[i,:] = np.reshape(EpochsT[i,:,:],(1,featureNum))
        FeaturesN = np.zeros((NumN, featureNum))
        for j in range(NumN):
            FeaturesN[j,:] = np.reshape(EpochsN[j,:,:],(1,featureNum))
        return [FeaturesT,FeaturesN]
    
def Standardization(Epochs):
    for i in range(Epochs.shape[1]):
        Epochs[:,i,:] = np.subtract(Epochs[:,i,:], np.mean(Epochs[:,i,:]))
        Epochs[:,i,:] = Epochs[:,i,:] / np.std(Epochs[:,i,:])
    
    return Epochs

def Make_Average_Component(EpochsT, NumT, EpochsN, NumN, channelNum, epochSampleNum, componentNum):
#     EpochsT = Standardization(EpochsT)
#     EpochsN = Standardization(EpochsN)
    
    NumT_Aver = NumT-componentNum
    NumN_Aver = NumN-componentNum
    
    EpochsT_Aver = np.zeros((NumT_Aver, channelNum, epochSampleNum))
    EpochsN_Aver = np.zeros((NumN_Aver, channelNum, epochSampleNum))
    for i in range(NumT_Aver):
        EpochsT_Aver[i, :, :] = np.mean(EpochsT[i:i+componentNum, :, :], axis=0)
    for j in range(NumN_Aver):
        EpochsN_Aver[j, :, :] = np.mean(EpochsN[j:j+componentNum, :, :], axis=0)
        
    return [EpochsT_Aver, NumT_Aver, EpochsN_Aver, NumN_Aver]

def resampling(Epochs, EpochNum, resampleRate, channelNum):
        resampled_epoch = np.zeros((EpochNum, channelNum, resampleRate))
        for i in range(EpochNum):
            for j in range(channelNum):
                resampled_epoch[i,j,:] = signal.resample(Epochs[i,j,:], resampleRate)
        return resampled_epoch
    
def main():
        start = time.time()
        
        ##Generate Preprocessing Training data
        ctime = datetime.today().strftime("%m%d_%H%M")
        Classifier_path = 'C:\\Users\\hyuns\\Desktop\\캡스톤\\LDAModel\\New\\' + ctime + 'Classifier.pickle'
        
        mat_path = root_path + 'Training\\'
        current_list = sorted(glob.glob(mat_path + '*.mat'), key=os.path.getmtime)
        
        matfile_name = current_list[0]
        
        mat = hdf5storage.loadmat(matfile_name)
        channelNames = mat['channelNames']
        eegData = mat['eegData']
        samplingFreq = mat['samplingFreq']
        samplingFreq = samplingFreq[0,0]
        stims = mat['stims']
        channelNum = channelNames.shape
        channelNum = channelNum[1]
        eegData = np.transpose(eegData)
        
        ##Preprocessing process
        
        #Bandpass Filter
        eegData = butter_bandpass_filter(eegData, low, high, samplingFreq, order=4)
    
        #Epoching
        epochSampleNum = int(np.floor(1.0 * samplingFreq))
        offset = int(np.floor(0.0 * samplingFreq)) 
        baseline = int(np.floor(1.0 * samplingFreq)) 
        [EpochsT, NumT] = Epoching(eegData, stims, 1, samplingFreq, channelNum, epochSampleNum, offset, baseline)
        [EpochsN, NumN] = Epoching(eegData, stims, 0, samplingFreq, channelNum, epochSampleNum, offset, baseline)
        
        EpochsN_New = np.zeros((NumT, channelNum, epochSampleNum))
        NumN = NumT
        for i in range(NumN):
            EpochsN_New[i,:,:] = np.mean(EpochsN[i*5:i*5+5, :, :], axis=0)
        
        EpochsN = EpochsN_New

        #Convert to feature vector
        [EpochsT, NumT, EpochsN, NumN] = Make_Average_Component(EpochsT, NumT, EpochsN, NumN, channelNum, epochSampleNum, 20)
        EpochsT = resampling(EpochsT, NumT, resampleRate, channelNum) 
        EpochsN = resampling(EpochsN, NumN, resampleRate, channelNum)
        
        featureNum = channelNum*resampleRate
        
        [FeaturesT, FeaturesN] = Convert_to_featureVector(EpochsT, NumT, EpochsN, NumN, featureNum)
        TrainData = np.concatenate((FeaturesT, FeaturesN))
        TrainLabel = np.concatenate((np.ones((NumT,1)).astype(int),np.zeros((NumN,1)).astype(int))).ravel()

        #Saving LDA classifier
        lda = LinearDiscriminantAnalysis(solver='lsqr',shrinkage='auto')
        lda.fit(TrainData, TrainLabel)
        joblib.dump(lda, Classifier_path, protocol=2)
        
        print("time :", time.time() - start)
        
if __name__ == "__main__":
    main()

C:\Users\hyuns\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


time : 1.447261095046997


In [4]:
import numpy as np
from scipy.signal import butter, lfilter, sosfiltfilt
import time
import os, glob
import hdf5storage
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.externals import joblib
import shutil
from datetime import datetime
import socket
from scipy import signal
import matplotlib.pyplot as plt

def Re_referencing(eegData, channelNum, sampleNum):
        after_car = np.zeros((channelNum,sampleNum))
        for i in np.arange(channelNum):
            after_car[i,:] = eegData[i,:] - np.mean(eegData,axis=0)
        return after_car
    
def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

def Standardization(Epochs):
    for i in range(Epochs.shape[1]):
        Epochs[:,i,:] = np.subtract(Epochs[:,i,:], np.mean(Epochs[:,i,:]))
        Epochs[:,i,:] = Epochs[:,i,:] / np.std(Epochs[:,i,:])
    
    return Epochs    
    
def Epoching(eegData, stims, code, samplingRate, nChannel, epochSampleNum, epochOffset,baseline):
        Time = stims[np.where(stims[:,1] == code),0][0]
        Time = np.floor(np.multiply(Time,samplingRate)).astype(int)
        Time_after = np.add(Time,epochOffset).astype(int)
        
        
        base = -90
        Time_base = np.add(Time,base).astype(int)
#         Time_base = np.add(Time,baseline).astype(int)
        Num = Time.shape
        Epochs = np.zeros((Num[0], nChannel, epochSampleNum))
        for j in range(Num[0]):
            Epochs[j, :, :] = eegData[:, Time_after[j]:Time_after[j] + epochSampleNum]
            
            for i in range(nChannel):
#                 Epochs[j,i,:] = Epochs[j,i,:] - np.mean(Epochs[j,i,:])
                Epochs[j,i,:] = Epochs[j,i,:] - np.mean(eegData[:, Time_base[j]:Time[j]])
    
    
#         Epochs = Standardization(Epochs)
        Epochs_Aver = np.mean(Epochs, axis=0)

        return Epochs_Aver
    
def Convert_to_FeatureVector(Epochs, buttonNum, featureNum):
    Features = np.zeros((buttonNum, featureNum))
    for i in range(buttonNum):
        Features[i, :] = np.reshape(Epochs[i, :, :], (1, featureNum))
    return Features

def resampling(Epochs, resampleRate, channelNum):
        resampled_epoch = np.zeros((channelNum, resampleRate))
        for j in range(channelNum):
            resampled_epoch[j,:] = signal.resample(Epochs[j,:], resampleRate)
            
        return resampled_epoch
    
def main():
        Classifier_path = 'C:\\Users\\hyuns\\Desktop\\캡스톤\\LDAModel\\New\\'
        classifier_list = sorted(glob.glob(Classifier_path + '*.pickle'), key=os.path.getmtime, reverse=True)
        print("classifer:", classifier_list[0])
        
        lda = joblib.load(classifier_list[0])
        
        mat_path = root_path + 'Online\\'
        current_list = sorted(glob.glob(mat_path + '*.mat'), key=os.path.getmtime)
        first_score = second_score = 0
        wrong_ans = []
        
        target_val = nontarget_val = 0
        count = 0
        for mat_file in current_list:
            print(mat_file)
            ans = mat_file[-5]
            
            mat = hdf5storage.loadmat(mat_file)
            channelNames = mat['channelNames']
            eegData = mat['eegData']
            samplingFreq = mat['samplingFreq']
            samplingFreq = samplingFreq[0,0]
            stims = mat['stims']
            channelNum = channelNames.shape
            channelNum = channelNum[1]
            eegData = np.transpose(eegData)
            buttonNum = 7
            
            #Bandpass Filter
            eegData = butter_bandpass_filter(eegData, low, high, samplingFreq, order=4)

            #Epoching
            epochSampleNum = int(np.floor(1.0 * samplingFreq))
            offset = int(np.floor(0.0 * samplingFreq))
            baseline = int(np.floor(1.0 * samplingFreq))
            
            ####### averaging whole epochs
            Epochs_Aver = np.zeros((buttonNum, channelNum, epochSampleNum))
            Epochs_final = np.zeros((buttonNum, channelNum, resampleRate))
            
            featureNum = channelNum*resampleRate
            
            for i in range(buttonNum):
                Epochs_Aver[i] = Epoching(eegData, stims, (i+1), samplingFreq, channelNum, epochSampleNum, offset, baseline)
                Epochs_final[i] = resampling(Epochs_Aver[i], resampleRate, channelNum)
            
            Features = Convert_to_FeatureVector(Epochs_final, buttonNum, featureNum)
        
            Answers = lda.decision_function(Features)
#             w_norm = np.linalg.norm(lda.coef_)
#             dist = Answers / w_norm
#             print(dist)
            
            for i in range(buttonNum):
                if i == int(ans)-1:
                    target_val = target_val + Answers[i]
                else:
                    nontarget_val = nontarget_val + Answers[i]
            
            answer = np.argmax(Answers) + 1
            
            print(Answers)
            if int(ans) == int(answer):
                if count > len(current_list)/2 - 1:
                    second_score += 1
                else:
                    first_score += 1
            else:
                wrong_ans.append(mat_file[mat_file.rfind('\\')+1:mat_file.rfind('_')])
            print('order: ', ans, 'predict: ', answer)
            
            count += 1
            
        print('wrong answer', wrong_ans)
        print('first score:', first_score, '/', int(len(current_list)/2))
        print('second score:', second_score, '/', int(len(current_list)/2))

        print('target mean:', target_val / len(current_list))
        print('nontarget mean:', nontarget_val / (len(current_list)*6))
    
if __name__ == "__main__":
    main()

classifer: C:\Users\hyuns\Desktop\캡스톤\LDAModel\New\0122_1408Classifier.pickle
C:\Users\hyuns\Desktop\캡스톤\EEGData\VR300\0729\hs\Online\49_5.mat
[-288.03253857 -297.03308888 -313.98880124 -362.44193783  287.65187997
 -425.9322275  -335.38216552]
order:  5 predict:  5
C:\Users\hyuns\Desktop\캡스톤\EEGData\VR300\0729\hs\Online\48_5.mat
[-247.28591674 -167.8949818  -328.42486588 -401.71934186  167.43082396
 -256.74629703 -179.35561172]
order:  5 predict:  5
C:\Users\hyuns\Desktop\캡스톤\EEGData\VR300\0729\hs\Online\47_1.mat
[  49.94177612 -238.53586295 -301.29973053 -293.54465678 -137.08927113
 -468.00171216 -398.83577861]
order:  1 predict:  1
C:\Users\hyuns\Desktop\캡스톤\EEGData\VR300\0729\hs\Online\46_7.mat
[-274.85751756 -268.15173988 -272.55799376 -419.10561254 -369.91293265
 -262.85536669  234.98675012]
order:  7 predict:  7
C:\Users\hyuns\Desktop\캡스톤\EEGData\VR300\0729\hs\Online\45_6.mat
[-373.45494249 -392.35597458 -242.70839473 -255.29077166 -278.03029819
  109.26105444 -320.0768471 ]
orde

 -162.34145314 -394.84443666]
order:  1 predict:  1
C:\Users\hyuns\Desktop\캡스톤\EEGData\VR300\0729\hs\Online\3_6.mat
[-189.27851563 -249.67826206 -272.2033648  -324.74520339 -387.52727993
  159.60392978 -204.61664631]
order:  6 predict:  6
C:\Users\hyuns\Desktop\캡스톤\EEGData\VR300\0729\hs\Online\2_4.mat
[-332.26660592 -372.37827707 -551.02174289  444.40014733 -319.51174387
 -436.9509441  -312.30815286]
order:  4 predict:  4
C:\Users\hyuns\Desktop\캡스톤\EEGData\VR300\0729\hs\Online\1_2.mat
[-124.15068479  136.47066996 -426.97254516 -418.82214543 -359.8093612
 -297.9536224  -171.58853364]
order:  2 predict:  2
wrong answer []
first score: 24 / 24
second score: 25 / 24
target mean: 153.32929303630107
nontarget mean: -312.70591105467554
